In [1]:
import os
import json
import torch
import random
import numpy as np
import torch.nn as nn
from PIL import Image
from skimage import io
from collections import deque
import matplotlib.pyplot as plt
import torchvision.models as models
from torch.autograd import Variable
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
reinforcement_indexes = [136, 187, 246, 332, 405, 457, 522, 524, 548, 582, 637, 813, 819, 826, 827, 868, 1031, 1168, 1194, 1216, 1223, 1233, 1242, 1246, 1276, 1353, 1358, 1367, 1526, 1576, 1653, 1700, 1729, 1737, 1770, 1790, 1836, 1846, 1913, 1914, 1924, 1932, 1967, 2124, 2187, 2257, 2258, 2309, 2315, 2320, 2370, 2430, 2450, 2520, 2540, 2550, 2577, 2802, 2838, 2846, 2861, 2864, 2903, 2992, 3031, 3039, 3085, 3180, 3201, 3214, 3288, 3338, 3354, 3428, 3451, 3494, 3575, 3649, 3665, 3675, 3677, 3758, 3786, 3823, 3826, 3833, 3888, 3918, 3952, 3969, 4027, 4054, 4142, 4194, 4255, 4262, 4282]
reinforcement_indexes = reinforcement_indexes * 4

In [4]:
DATASET_DIR = "../../progetto_2021_dataset/"
UNLABELED_DIR = "../../unlabeled/"
CHECKPOINTS_DIR = "../../checkpoints/"

JSON_DATA = os.path.join(DATASET_DIR, 'train_test_split_dict.json')

with open(JSON_DATA) as fp:
    dataset_json = json.load(fp)

with open("../../unlabeled/unlabeled1_keys.json", "r") as fd:
    unlabeled_dirs = json.loads(json.load(fd))

FileNotFoundError: [Errno 2] No such file or directory: '../../progetto_2021_dataset/train_test_split_dict.json'

In [ ]:
class CustomVGG(nn.Module):

  def __init__(self):
    super(CustomVGG, self).__init__()

    self.vgg = models.vgg16(pretrained=True, progress=False)
    for param in self.vgg.features.parameters():
      param.requires_grad = False

    self.vgg.classifier.add_module("7", nn.Linear(1000, 85)) 

  def forward(self, data):
    return self.vgg(data)

In [ ]:
class Augmenter(object):
    def __init__(self, to_generate=30):
        self.to_generate = to_generate
        self.transformation = transforms.RandomChoice([
                                transforms.ColorJitter(brightness=1, contrast=1, saturation=0.5),
                                transforms.RandomRotation((10,180)),
                                transforms.RandomAffine(0, translate=(0.2, 0.2))
        ])

    def __call__(self, images):
        output = torch.zeros((self.to_generate, 3, 224, 224), dtype=torch.float32).to(device)
        output[: images.shape[0]] = images
        added_images = images.shape[0]
        while added_images < self.to_generate:
            rand_index = random.randint(0, images.shape[0] -1)
            img = images[rand_index].clone()
            output[added_images] = self.transformation(img)
            added_images += 1
        return output

In [ ]:
class Dispatcher(object):
    def __init__(self, device="cpu", criterion=nn.BCEWithLogitsLoss(), largest=False, k=30):
        super(Dispatcher, self).__init__()
        self.criterion = criterion
        self.largest = largest
        self.k = k
        
        self.vgg = CustomVGG().to(device)
        self.vgg.load_state_dict(torch.load("../../checkpoints/weighted_custom_vgg_10e.pth"))
        
        self.augmenter = Augmenter(k)

    def __call__(self, images, labels=None): #images = imm di un particolare trailer
        len_ = images.shape[0]

        if len_ > self.k:
            return self.select_data(images, labels)
        else:
            return self.augmenter(images)


    def select_data(self, images, labels):
        y_scores = self.vgg(images)
        if labels == None:
            top_k_indexes = torch.randint(images.shape[0], (self.k,))
        else:
            top_k_indexes = self.select_top_k(y_scores, labels[0])
        return images[top_k_indexes]


    def select_top_k(self, y_preds, y_true):
        n_items = y_preds.shape[0]
        scores = torch.zeros(n_items)
        for i in range(n_items):
            scores[i] = self.criterion(y_preds[i], y_true)
        top_k_indexes = torch.topk(scores, self.k, largest=self.largest)[1]
        return top_k_indexes.numpy()

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()

        resnet = models.resnet34(pretrained=True, progress=False) 
        
        layers = []
     
        layers.append(resnet.conv1)
        layers.append(resnet.bn1)
        layers.append(resnet.relu)
        layers.append(resnet.maxpool)
        layers.append(resnet.layer1)
        layers.append(resnet.layer2)
        layers.append(resnet.layer3)
        layers.append(resnet.layer4)

        self.feature_extraction = nn.Sequential(*layers)

    def forward(self, samples):
        return self.feature_extraction(samples)

In [ ]:
class BaggingSummarization(nn.Module):
    def __init__(self):
        super(BaggingSummarization, self).__init__()

        self.pooling = nn.AdaptiveAvgPool2d(output_size=(1, 1))


    def forward(self, samples):
        output = self.pooling(samples)
        return output.view(-1, 512)

In [ ]:
class BaggingClassifier(nn.Module):
    def __init__(self, input_shape):
        super(BaggingClassifier, self).__init__()

        self.summarization = BaggingSummarization()

        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.Dropout(0.4),
            nn.Linear(128, 85)
        )

    def forward(self, samples):
        output_summaritazion = self.summarization(samples)
        classifier_scores = self.classifier(output_summaritazion)
        return torch.div(torch.sum(classifier_scores, axis=0), 30.0).unsqueeze(0)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dim=1024):
        super(Encoder, self).__init__()


        self.encoder_end = nn.Sequential(
            nn.Flatten(),
            nn.BatchNorm1d(25088),
            nn.Linear(25088, 2048),
            nn.Dropout(0.4),
            nn.Linear(2048, latent_dim)
        )    


    def forward(self, samples):
        return self.encoder_end(samples)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=1024):
        super(Decoder, self).__init__()

        self.linears = nn.Sequential(
                                        nn.Linear(latent_dim, 2048),
                                        nn.Dropout(0.4),
                                        nn.Linear(2048, 25088)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1), # 14
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1), # 28
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1), # 56
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1), # 112
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), #224
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(16, 3, 3, padding=1), # 224
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True)
        )


    def forward(self, samples):
        samples = self.linears(samples)
        samples = samples.view(samples.shape[0], 512, 7, 7)
        return self.decoder(samples)

In [ ]:
class Architecture(nn.Module):
    
    def __init__(self, device="cpu", k=30):
        super(Architecture, self).__init__()
        
        self.executor = ThreadPoolExecutor(max_workers=1)

        self.dispatcher = Dispatcher(device=device, k=k)

        self.feature_extraction = CustomResNet().to(device)

        self.classifier = BaggingClassifier((k, 512, 7, 7)).to(device)

        self.encoder = Encoder().to(device)

        self.decoder = Decoder().to(device)


    def forward(self, samples, labels):
        samples = self.dispatcher(samples, labels)
        
        features = self.feature_extraction(samples)
    
        task = self.executor.submit(lambda: self.decoder(self.encoder(features)))
    
        output_classifier = self.classifier(features)
       
        output_decoder = task.result()
        return samples, output_classifier, output_decoder

In [ ]:
architecture = Architecture(device, 30)

In [ ]:
labels = set()

for k in dataset_json.values():
    for lable_list in k.values():
        for v in lable_list:
            labels.add(v)
            
label_idx = {v: i for i, v in enumerate(sorted(labels))}

In [ ]:
class SourceDataset(Dataset):

    def __init__(self, path, dataset_dict, labels_int_dict, tranformations=None):
      self.path = path
      self.dataset_dict = dataset_dict
      self.tranformations = transforms.ToTensor() if tranformations is None else tranformations
      
      self.to_skip = self.clean__()

      to_keep = set(self.dataset_dict.keys()).difference(self.to_skip)

      self.dirs_ = np.array(list(to_keep))
      self.labels_dict_ = labels_int_dict

      self.len = self.dirs_.shape[0]
      self.labels_len = len(self.labels_dict_)


    def clean__(self):
      to_skip = []
      for dir_name in self.dataset_dict.keys():
        dir = self.path + "/" + dir_name
        if os.path.isdir(dir):
          if len(os.listdir(dir)) == 0:
            to_skip.append(dir)
      
      return set(to_skip)


    def __len__(self):
        return self.len

    
    def __getitem__(self, idx):
        name = self.dirs_[idx]
        folder_pattern = os.path.join(self.path, name, '*.png')
        images = io.imread_collection(folder_pattern)
        data = torch.zeros(len(images), 3, 224, 224)
        
        for i, image in enumerate(images):
          image = self.tranformations(Image.fromarray(image))
          data[i] = image
        labels = torch.zeros(self.labels_len, dtype=torch.float32)
        for label in self.dataset_dict[name]:
          labels[self.labels_dict_[label]] = 1
        
        return data, labels

In [ ]:
class MemorySourceDataset(Dataset):

    def __init__(self, path, dataset_dict, labels_int_dict, tranformations=None):

      self.path = path
      self.dataset_dict = dataset_dict
      self.tranformations = transforms.ToTensor() if tranformations is None else tranformations
      
      self.to_skip = self.clean__()

      to_keep = set(self.dataset_dict.keys()).difference(self.to_skip)

      self.dirs_ = np.array(list(to_keep))
      self.labels_dict_ = labels_int_dict

      self.len = self.dirs_.shape[0]
      self.labels_len = len(self.labels_dict_)

      self.data = self.memload__()


    def memload__(self, jobs=6):
        executor = ThreadPoolExecutor(max_workers=jobs)

        jobs = []
        
        intervals = [(0, 715), (715, 1430), (1430, 2145), (2145, 2860), (2860, 3575), (3575, 4292)]
        i = 0
        for (start, end) in intervals:
            print(f"Job{i} submitted")
            jobs.append(executor.submit(self.range_load__, start, end))
            i += 1
        
        data = []
        for job in jobs:
            data = data + job.result()

        executor.shutdown()
        
        return data
    
    def range_load__(self, start, end):
        data = []
        for i in range(start, end):
            data.append(self.dskload__(i))
        return data
            

    def clean__(self):
      to_skip = []
      
      for dir_name in self.dataset_dict.keys():
        dir = self.path + "/" + dir_name
        if os.path.isdir(dir):
          if len(os.listdir(dir)) == 0:
            to_skip.append(dir)
      
      return set(to_skip)

    
    def dskload__(self, idx):
        name = self.dirs_[idx]
        folder_pattern = os.path.join(self.path, name, '*.png')
        images = io.imread_collection(folder_pattern)
        data = torch.zeros(len(images), 3, 224, 224)
        
        for i, image in enumerate(images):
          image = self.tranformations(Image.fromarray(image))
          data[i] = image
        labels = torch.zeros(self.labels_len, dtype=torch.float32)
        for label in self.dataset_dict[name]:
          labels[self.labels_dict_[label]] = 1
        
        return data, labels

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx]


In [ ]:
class UnlabeledSourceDataset(Dataset):

    def __init__(self, path, dir_names, transformations=None):
      self.path = path
      self.dir_names = dir_names
      self.transformations = transforms.ToTensor() if transformations is None else transformations
      
      self.to_skip = self.clean__()

      to_keep = set(self.dir_names).difference(self.to_skip)

      self.dirs_ = np.array(list(to_keep))

      self.len = self.dirs_.shape[0]


    def clean__(self):
      to_skip = []
      labels = set()
      for dir_name in self.dir_names:
        dir = self.path + "/" + dir_name
        if os.path.isdir(dir):
          if len(os.listdir(dir)) == 0:
            to_skip.append(dir)
      
      return set(to_skip)


    def __len__(self):
        return self.len

    
    def __getitem__(self, idx):
        name = self.dirs_[idx]
        folder_pattern = os.path.join(self.path, name, '*.png')
        images = io.imread_collection(folder_pattern)
        data = torch.zeros(len(images), 3, 224, 224)
        
        for i, image in enumerate(images):
          image = self.transformations(Image.fromarray(image))
          data[i] = image
        
        return data

In [ ]:
transformations = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Resize((240, 240)),
                                        transforms.CenterCrop((224, 224)),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ])
train_dataset = MemorySourceDataset(DATASET_DIR, dataset_json["train"], label_idx, transformations)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = 0.75
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * ((1-pt) ** self.gamma) * BCE_loss
        return F_loss.mean()

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.focal_loss = FocalLoss()
        self.mse = nn.MSELoss()


    def forward(self, y_pred, y_true, reconstructed, images):
        focal_loss_score = self.focal_loss(y_pred, y_true)
        mse_score = self.mse(reconstructed, images)
        combined_loss = focal_loss_score + mse_score * 0.1
        return combined_loss

In [ ]:
combined_loss = CombinedLoss()

In [ ]:
learning_rate = 0.0005
optimizer = torch.optim.Adam(architecture.parameters(), lr=learning_rate, weight_decay=1e-5)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=1, 
                                           shuffle=True)

In [ ]:
def save_train_checkpoint(epoch, model, optimizer, loss, name, root=CHECKPOINTS_DIR):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, os.path.join(root, name))

In [ ]:
TRAIN_CHECKPOINT_NAME = "arch_bce_reinforced_10e.pth" #TODO
MODEL_CHECKPOINT_NAME = "arch_bce_reinforced_10e.pth"

In [ ]:
mask = torch.tensor([-1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,
                    -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
                    -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
                    1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
                    1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
                    1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,
                    1.])

In [ ]:
trasformations_data_augmentation = transforms.RandomChoice([
                                                                transforms.RandomRotation([-13,+13]),
                                                                transforms.RandomHorizontalFlip(1),
                                                                transforms.RandomVerticalFlip(1),
                                                                transforms.RandomAffine(0, translate=(0.2, 0.2)),
                                                                transforms.RandomAffine(degrees=0,scale=(0.5, 0.75))
                                                            ]) 

In [ ]:
def transform_images(samples, transformation=trasformations_data_augmentation):
    result = torch.zeros_like(samples)
    for i in range(samples.shape[0]):
        result[i] = trasformations_data_augmentation(samples[i])
    return result

In [ ]:
def reinforced_train(model, combined_loss, optimizer, loader, train_set, mask, epochs=5, threshold=2, device="cpu"):
    total_steps = len(loader)
    loss = 0
    model = model.to(device)
    mask = mask.to(device)
    for epoch in range(1, epochs + 1):
        random.shuffle(reinforcement_indexes)
        queue_indexes = deque(reinforcement_indexes)
        for batch_idx, (images, labels) in enumerate(loader):  
            images = images[0].to(device)
            labels = labels.to(device)
            images, out_clf, out_dec = model(images, labels)
            loss = combined_loss(out_clf, labels, out_dec, images)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            if batch_idx % 200 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, epochs, batch_idx, total_steps, loss.item()))
            
            if (batch_idx + 1) % 11 == 0 and len(queue_indexes) > 0: 
                augmented_images = transform_images(train_set[queue_indexes.pop()][0]).to(device)
                augmented_images, out_clf, out_dec = model(augmented_images, labels)
                loss = combined_loss(out_clf, labels, out_dec, augmented_images)
                optimizer.zero_grad() 
                loss.backward()
                optimizer.step()
        name = "re_" + TRAIN_CHECKPOINT_NAME
        save_train_checkpoint(epoch, model, optimizer, loss, name)

In [ ]:
def reinforced_train_with_unlabeled(model, combined_loss, optimizer, loader, unlabeled_loader, train_set, mask, epochs=5, threshold=2, step=3, top_k=10, device="cpu"):
    total_steps = len(loader)
    loss = 0
    model = model.to(device)
    mask = mask.to(device)
    unlabeled_iterator = iter(unlabeled_loader) 
    for epoch in range(1, epochs + 1):
        random.shuffle(reinforcement_indexes)
        queue_indexes = deque(reinforcement_indexes)
        for batch_idx, (images, labels) in enumerate(loader):  
            images = images[0].to(device)
            labels = labels.to(device)
            images, out_clf, out_dec = model(images, labels)
            loss = combined_loss(out_clf, labels, out_dec, images)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            if batch_idx % 200 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, epochs, batch_idx, total_steps, loss.item()))
            
            if (batch_idx + 1) % 11 == 0 and len(queue_indexes) > 0: 
                augmented_images = transform_images(train_set[queue_indexes.pop()][0]).to(device)
                augmented_images, out_clf, out_dec = model(augmented_images, labels)
                loss = combined_loss(out_clf, labels, out_dec, augmented_images)
                optimizer.zero_grad() 
                loss.backward()
                optimizer.step()

            if (batch_idx + 1) % step == 0:
                images = next(unlabeled_iterator) 
                images = images[0].to(device) 
                images, out_clf, out_dec = model(images, None) 
                y_pred = torch.sigmoid(out_clf)
                _, idx = torch.topk(y_pred, top_k, dim=1) 
                y_pred = torch.zeros_like(y_pred) 
                y_pred.scatter_(1, idx, 1) 
                loss = combined_loss(out_clf, y_pred, out_dec, images) 
 
                optimizer.zero_grad()  
                loss.backward() 
                optimizer.step()
                
        name = "re_" + TRAIN_CHECKPOINT_NAME
        save_train_checkpoint(epoch, model, optimizer, loss, name)

In [ ]:
def save_model(model, name, root=CHECKPOINTS_DIR):
  torch.save(model.state_dict(), 
             os.path.join(root, name))

def load_model(name, root=CHECKPOINTS_DIR):
  return torch.load(os.path.join(root, name))


def exists_checkpoint(name, root=CHECKPOINTS_DIR):
  return os.path.isfile(os.path.join(root, name))

In [ ]:
def train(model, combined_loss, optimizer, loader, epochs, device="cpu"):
    total_steps = len(loader)
    loss = 0
    model = model.to(device)
    for epoch in range(1, epochs + 1):
        for batch_idx, (images, labels) in enumerate(loader):  
            images = images[0].to(device)
            labels = labels.to(device)
            images, out_clf, out_dec = model(images, labels)
            loss = combined_loss(out_clf, labels, out_dec, images)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            if batch_idx % 200 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, epochs, batch_idx, total_steps, loss.item()))
        save_train_checkpoint(epoch, model, optimizer, loss, TRAIN_CHECKPOINT_NAME)      

In [ ]:
exists_checkpoint(MODEL_CHECKPOINT_NAME)

In [ ]:
if not exists_checkpoint(MODEL_CHECKPOINT_NAME):
    train(architecture, combined_loss, optimizer, train_loader, epochs=5, device=device)
    save_model(architecture, MODEL_CHECKPOINT_NAME)
    reinforced_train(architecture, combined_loss, optimizer, train_loader, 10, mask, threshold=3, times=3, device=device)
    save_model(architecture, "re_" + MODEL_CHECKPOINT_NAME)
else:
    print("Checkpoint Found!")
    architecture.load_state_dict(load_model(MODEL_CHECKPOINT_NAME))

In [ ]:
reinforced_train(architecture, combined_loss, optimizer, train_loader, train_dataset, mask, epochs=3, threshold=2, device=device)

In [ ]:
train(architecture, combined_loss, optimizer, train_loader, epochs=2, device=device)
save_model(architecture, MODEL_CHECKPOINT_NAME)

In [ ]:
reinforced_train(architecture, combined_loss, optimizer, train_loader, 5, mask, threshold=3, times=3, device=device)
save_model(architecture, "re_" + MODEL_CHECKPOINT_NAME)

In [ ]:
unlabeled_dataset = UnlabeledSourceDataset(UNLABELED_DIR, unlabeled_dirs, transformations)

In [ ]:
unlabeled_loader = torch.utils.data.DataLoader(dataset=unlabeled_dataset, 
                                               batch_size=1,
                                               shuffle=True)

In [ ]:
def unlabeled_train(model, combined_loss, optimizer, labeled_loader, unlabeled_loader, epochs, top_k=10, device="cpu"):
    total_steps = len(labeled_loader) 
    loss = 0 
    step = 4 
    model = model.to(device) 
    unlabeled_iterator = iter(unlabeled_loader) 
    for epoch in range(1, epochs + 1): 
        for batch_idx, (images, labels) in enumerate(labeled_loader): 
            images = images[0].to(device) 
            labels = labels.to(device) 
            images, out_clf, out_dec = model(images, labels) 
            loss = combined_loss(out_clf, labels, out_dec, images) 
  
            optimizer.zero_grad()  
            loss.backward() 
            optimizer.step() 
 
            if (batch_idx + 1) % step == 0: 
                images = next(unlabeled_iterator) 
                images = images[0].to(device) 
                images, out_clf, out_dec = model(images, None) 
                y_pred = torch.sigmoid(out_clf)
                _, idx = torch.topk(y_pred, top_k, dim=1) 
                y_pred = torch.zeros_like(y_pred) 
                y_pred.scatter_(1, idx, 1) 
                loss = combined_loss(out_clf, y_pred, out_dec, images) 
 
                optimizer.zero_grad()  
                loss.backward() 
                optimizer.step() 
 
            if batch_idx % 200 == 0: 
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, epochs, batch_idx, total_steps, loss.item())) 
        save_train_checkpoint(epoch, model, optimizer, loss, "arch_focal_loss_ext_ul_train_11e.pth")

In [ ]:
unlabeled_train(architecture, combined_loss, optimizer, train_loader, unlabeled_loader, epochs=2, top_k=10, device=device)

In [ ]:
transformations = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Resize((240, 240)),
                                        transforms.CenterCrop((224, 224)),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ])
                                    
test_dataset = SourceDataset(DATASET_DIR, dataset_json["test"], label_idx, transformations)

In [ ]:
reinforced_train_with_unlabeled(architecture, combined_loss, optimizer, train_loader, unlabeled_loader, train_dataset, mask, epochs=3, threshold=2, step=3, top_k=10, device=device)

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1)

In [ ]:
def test(model, criterion, loader, labels_idx, topk=10, device="cpu"):
    test_loss = 0
    correct = 0
    predictions = []
    y_true = []
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images[0].to(device)
            labels = labels.to(device)
            model = model.to(device)
            images, out_clf, out_dec = model(images, labels)
            
            loss = criterion(out_clf, labels, out_dec, images)
            y_pred = torch.sigmoid(out_clf)
            _, idx = y_pred.topk(topk, dim=1)
            y_pred = torch.zeros_like(y_pred)
            y_pred.scatter_(1, idx, 1)
            predictions.append(y_pred.cpu())
            y_true.append(labels.cpu())
            if batch_idx % 200 == 0:
                print(batch_idx)
        y_true, predictions = torch.cat(y_true, axis=0), torch.cat(predictions, axis=0)
        report = classification_report(y_true, predictions, target_names=list(sorted(labels_idx.keys())))
        print(report)
        return y_true, predictions

In [ ]:
y_true, predictions = test(architecture, combined_loss, test_loader, label_idx, device=device)

In [ ]:
reinforced_train(architecture, combined_loss, optimizer, train_loader, train_dataset, mask, epochs=1, threshold=2, device=device)

In [ ]:
reinforced_train_with_unlabeled(architecture, combined_loss, optimizer, train_loader, unlabeled_loader, train_dataset, mask, epochs=2, threshold=2, step=3, top_k=10, device=device)

In [ ]:
reinforced_train(architecture, combined_loss, optimizer, train_loader, train_dataset, mask, epochs=1, threshold=2, device=device)

In [ ]:
y_true, predictions = test(architecture, combined_loss, test_loader, label_idx, device=device)

In [ ]:
save_model(architecture, "16_56_25.pth")

In [ ]:
architecture = Architecture(device=device, k=30)

In [ ]:
architecture.load_state_dict(load_model("16_56_25.pth"))

In [ ]:
optimizer = torch.optim.Adam(architecture.parameters(), lr=learning_rate, weight_decay=1e-5)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=1, 
                                           shuffle=True)

In [ ]:
reinforced_train_with_unlabeled(architecture, combined_loss, optimizer, train_loader, unlabeled_loader, train_dataset, mask, epochs=1, threshold=2, step=3, top_k=10, device=device)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=1, 
                                           shuffle=True)

In [ ]:
train(architecture, combined_loss, optimizer, train_loader, epochs=2, device=device)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=1, 
                                           shuffle=True)

In [ ]:
reinforced_train(architecture, combined_loss, optimizer, train_loader, train_dataset, mask, epochs=1, threshold=2, device=device)

In [ ]:
test_loader = torch.utils.data. DataLoader(dataset=test_dataset, batch_size=1)
y_true, predictions = test(architecture, combined_loss, test_loader, label_idx, device=device)